In [445]:
#import

import bs4
import json
import time
import requests
import scipy as sc
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

driver=webdriver.Chrome()


starting to build our dataset using selenium

the contest started in the year 1956 BUT only at 1957 they started to rank with points so this is where we'll start

Scrolling through pages in both sites

In [446]:

def move_page_euroWorld(Syear):
    
    if Syear==2020:
        driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

    driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

In [447]:

# def move_page_eurovision(Syear):
#     driver.get("https://eurovision.tv/events")
#     time.sleep(6)
#     driver.find_element_by_xpath('//*[@id="onetrust-reject-all-handler"]').click()
#     # driver.find_element_by_xpath('/html/body/a').click()
#     driver.find_element(By.CSS_SELECTOR,f"[href*='{Syear}']").click()
#     driver.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/div[2]/a[3]').click()
#     driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div[1]/nav/div/a[2]').click()


In [448]:
#lists to crawl from eurovisionworld generalBoard
Syear=1975
place=[]
country=[]
song=[]
artist=[]
year=[]
point=[]
normPoints=[]
normelized=[]
running=[]
contestLoc=[]
# from eurovision official site:

one=[]
two=[]
three=[]
four=[]
five=[]
six=[]
seven=[]
eight=[]
ten=[]
twelve=[]
points_year=[]

#from wikipedia
vote_array=[]
vote_dict={}

in this part we will crawl from eurovisionworld in order to recive the following: place, country,song,artist,year,point,running


In [449]:
driver.get("https://eurovisionworld.com/eurovision/1975")
while Syear<=1975:
    location=driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[1]/span/span/a').text.split(' ')[0]
    #taking data from the general board
    general_Table =driver.find_element_by_xpath('//*[@id="voting_table"]/table/tbody')
    #from the table on EurovisionWorld 
    for row in general_Table.find_elements_by_css_selector('tr'):
        place.append(row.find_elements_by_tag_name('td')[0].text)
        country.append(row.find_elements_by_tag_name('td')[1].text)
        #spliting the song and artist from the same cell:
        song.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[0])
        artist.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[1])
        point.append(row.find_elements_by_tag_name('td')[3].text)
        if Syear>=2016:
            running.append(row.find_elements_by_tag_name('td')[6].text)
        else:
            running.append(row.find_elements_by_tag_name('td')[4].text)
        year.append(Syear)
        contestLoc.append(location)
    
    scoring_Table=driver.find_element_by_xpath('//*[@id="scoreboard"]/div/div[1]/table/tbody')
    table_head=driver.find_element_by_xpath('//*[@id="scoreboard"]/div/div[1]/table/thead/tr')

    i=0
    while i<len(year):
        for country_name in table_head.find_elements_by_css_selector('td'):
            for row in scoring_Table.find_elements_by_css_selector('tr'):
                vote_array.append(row.find_elements_by_tag_name('td')[i].text)
            vote_dict[country_name.find_element_by_xpath(f'//*[@id="scoreboard"]/div/div[1]/table/thead/tr/td[{2+i}]').get_attribute('data-from')]=vote_array
            vote_array=[]
            i+=1


    Syear+=1
    move_page_euroWorld(Syear)
    if Syear==2020:
        Syear+=1

C:\Users\A0D5~1\AppData\Local\Temp/ipykernel_8136/1359015758.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  location=driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[1]/span/span/a').text.split(' ')[0]
C:\Users\A0D5~1\AppData\Local\Temp/ipykernel_8136/1359015758.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  general_Table =driver.find_element_by_xpath('//*[@id="voting_table"]/table/tbody')
C:\Users\לירם\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\remote\webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\לירם\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="scoreboard"]/div/div[1]/table/thead/tr/td[21]"}
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x01256903+2517251]
	Ordinal0 [0x011EF8E1+2095329]
	Ordinal0 [0x010F2848+1058888]
	Ordinal0 [0x0111D448+1233992]
	Ordinal0 [0x0111D63B+1234491]
	Ordinal0 [0x01113AB1+1194673]
	Ordinal0 [0x0113650A+1336586]
	Ordinal0 [0x01113A36+1194550]
	Ordinal0 [0x011365BA+1336762]
	Ordinal0 [0x01145BBF+1399743]
	Ordinal0 [0x0113639B+1336219]
	Ordinal0 [0x011127A7+1189799]
	Ordinal0 [0x01113609+1193481]
	GetHandleVerifier [0x013E5904+1577972]
	GetHandleVerifier [0x01490B97+2279047]
	GetHandleVerifier [0x012E6D09+534521]
	GetHandleVerifier [0x012E5DB9+530601]
	Ordinal0 [0x011F4FF9+2117625]
	Ordinal0 [0x011F98A8+2136232]
	Ordinal0 [0x011F99E2+2136546]
	Ordinal0 [0x01203541+2176321]
	BaseThreadInitThunk [0x75D06739+25]
	RtlGetFullPathName_UEx [0x771D8AFF+1215]
	RtlGetFull

In [ ]:
print(vote_dict)

{'be': ['3', '10', '6', '2', '', '8', '', '', '12', '4', '1', '7', '5', '', '', '', '', '', ''], 'fi': ['10', '7', '12', '', '5', '4', '', '6', '1', '8', '3', '', '', '2', '', '', '', '', ''], 'fr': ['5', '12', '4', '', '3', '10', '', '7', '6', '', '1', '8', '', '', '', '2', '', '', ''], 'de': ['8', '10', '4', '', '', '6', '12', '', '', '5', '1', '', '2', '3', '7', '', '', '', ''], 'ie': ['8', '3', '6', '12', '10', '2', '5', '', '', '7', '1', '', '4', '', '', '', '', '', ''], 'il': ['12', '10', '5', '7', '6', '', '8', '3', '', '4', '', '1', '', '2', '', '', '', '', ''], 'it': ['1', '3', '', '8', '10', '12', '', '', '4', '6', '2', '', '', '5', '', '', '', '7', ''], 'lu': ['10', '12', '3', '', '', '2', '6', '7', '', '', '1', '5', '', '4', '', '', '8', '', ''], 'mt': ['12', '8', '10', '7', '5', '6', '', '2', '4', '', '', '', '', '1', '', '', '3', '', ''], 'mc': ['10', '12', '1', '7', '', '5', '8', '6', '', '4', '', '', '', '', '', '', '', '2', '3'], 'nl': ['', '4', '', '8', '12', '7', '',

In [ ]:
dict={'Country':country,'Song':song,'Artist':artist,'Year':year,'Points':point,'Running_Order':running,'Place':place,'Contest_Location':contestLoc }
df=pd.DataFrame(dict)
df["Points"] = pd.to_numeric(df["Points"], downcast="float")
print(vote_dict)
# df.to_csv('eurovision_dataset.csv')
# df

{}


Crawling throgh Eurovision site to find out the scoring
